In [28]:
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"/Users/kaka/Downloads/instantclient_18_1")
from sqlalchemy import create_engine
import json
import pandas as pd
import os
import sqlalchemy

ProgrammingError: Oracle Client library has already been initialized

In [2]:
def get_data_from_oracle(sql_query):
    username = 'nppbuf'  
    password = 'Svwnppbuf321'  
    hostname = '10.122.6.59'  
    port = 1521  
    service_name = 'b2bt'
    dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
# 连接 Oracle 数据库  
    connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)

    chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值  
    query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
# 遍历查询结果的每个批次，并将其合并为一个 DataFrame  
    result = pd.DataFrame()  
    for chunk in query_chunks: 
        result = result.append(chunk)
# 关闭数据库连接  
    connection.close()
# 输出合并后的 DataFrame  
    return result

In [3]:
# 主数据
raw_sql_query = """ 
SELECT * from nppbuf.T_DD_FORECAST_DETAIL_FJHIS  
where  HOSTPARTID  in （select 
       hostpartid
  from nppbuf.t_dd_part_chain_details
 where hostpartchainid not like '%RDC'
   AND hostpartid != hostchainparentid ） 
"""
raw_sql_info = get_data_from_oracle(raw_sql_query) 
raw_sql_info.columns

/Users/kaka/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_39693/2423816003.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(chunk)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_39693/2423816003.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(chunk)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_39693/2423816003.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Index(['HOSTPARTID', 'HOSTLOCID', 'HISTORYBEGDATE', 'DSHOSTID',
       'HISTORYAMOUNT', 'ABC_HITS_NETWORK', 'ABC_COST', 'ABC_DEMAND', 'AA',
       'BB', 'CC'],
      dtype='object')

In [4]:
raw_sql_info.columns = ['零件号', '仓库代码', '年月', '需求流', '需求数量', 'ABC需求频次分类',
                          'ABC价格分类', 'ABC需求数量分类', '零件号第四位','组装包','是否预测']

In [5]:
# 主数据
alt_replace_sql_query = """ 
   select substr(hostpartchainid, 1, 4) as host_locid, --工厂
       relationtype,                  --0代表Replace,1代表Alternate
       hostpartid,                    --被替代件
       hostchainparentid              --替代件
  from nppbuf.t_dd_part_chain_details
 where hostpartchainid not like '%RDC'
   AND hostpartid != hostchainparentid
"""
alt_replace_info = get_data_from_oracle(alt_replace_sql_query)   
alt_replace_info.columns =['工厂', '是否继续采购','零件号', '替代件']

/Users/kaka/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_39693/2423816003.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(chunk)


In [6]:
# 主数据
demods_counts_sql_query = """ 
     select substr(hostlocid, 1, 4), --工厂
       hostpartid, --零件代码
       onhandnew + AMOUNTCUSTOM1 + AMOUNTCUSTOM2 --库存
  from nppbuf.t_Dd_Stock_Amount
 where substr(hostlocid, 1, 1) = '1'
"""
demods_counts_info = get_data_from_oracle(demods_counts_sql_query) 
demods_counts_info.columns =['工厂','零件号', '库存']

/Users/kaka/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_39693/2423816003.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(chunk)


In [7]:
def process_element(element):  
    # 检查第二位是否为0  
    if element[1] == '0':  
        return 1000  
    else:  
        return 1100  
    
raw_sql_info['工厂'] = raw_sql_info['仓库代码'].apply(process_element)  
raw_sql_info['工厂'] = [str(x) for x in raw_sql_info['工厂']]
raw_sql_info_isalive = pd.merge(raw_sql_info,alt_replace_info,on = ['工厂','零件号'],how = 'left')

raw_sql_info_isalive_counts = pd.merge(raw_sql_info_isalive,demods_counts_info,
                                       on = ['工厂','零件号'],how = 'left')

raw_sql_info_isalive_counts['库存'].fillna(0,inplace =True)

In [8]:
#   011，032 = Replace ==0
# 非011，032 = Alternate ==1  那么CPD和RDC都会继续采购/补充老零件
# raw_sql_info_isalive_counts[raw_sql_info_isalive_counts['是否继续采购']!=1]

In [9]:
# 使用apply函数根据条件更新零件号列
def update_part_number(row):
    if row['是否继续采购']==1:
        return row['零件号']
    else:
        if row['库存']>0:
            return row['零件号']
        else:
            return row['替代件']

raw_sql_info_isalive_counts['零件号_new'] = raw_sql_info_isalive_counts.apply(update_part_number, axis=1)

# 最终的DataFrame merged_df 包含了修正后的零件号


In [56]:
raw_sql_info_isalive_counts.shape

(2936273, 16)

In [11]:
raw_sql_info_isalive_counts[raw_sql_info_isalive_counts['零件号']!=raw_sql_info_isalive_counts['零件号_new']]

,零件号,仓库代码,年月,需求流,需求数量,ABC需求频次分类,ABC价格分类,ABC需求数量分类,零件号第四位,组装包,是否预测,工厂,是否继续采购,替代件,库存,零件号_new
34,5QD129607D,6000,20230531,DEALER,1,A,C,ZERO,1,N,Y,1000,0,5QD129607K,0.0,5QD129607K
35,3CN864759,1000-1,20230531,DEALER,1,None,None,None,8,N,Y,1000,0,3CG864759,0.0,3CG864759
143,1JD698451A,1000-1,20230531,DEALER,2,None,None,None,6,N,Y,1000,0,1JD698451B,0.0,1JD698451B
162,1JD698451A,1000-1,20230531,DEALER,1,None,None,None,6,N,Y,1000,0,1JD698451B,0.0,1JD698451B
198,096323981D,1000-1,20220912,DEALER,1,None,None,None,3,N,Y,1000,0,096323981R,0.0,096323981R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936267,03F907660E,1000-3,20220125,DEALER,1,A,C,ZERO,9,N,Y,1000,0,06L907660C,0.0,06L907660C
2936268,04E121600BJ,1000-1,20220813,DEALER,1,None,None,None,1,N,Y,1000,0,04E121600BL,0.0,04E121600BL
2936269,101905601E,8023,20220108,DEALER,20,A,C,ZERO,9,N,Y,1000,0,101905601B,0.0,101905601B
2936271,04E115561H,6000,20220716,DEALER,10,A,C,ZERO,1,N,Y,1000,0,04E115561L,0.0,04E115561L


In [12]:
load_data  = raw_sql_info_isalive_counts[['零件号_new','仓库代码', '年月', '需求流', '需求数量', 'ABC需求频次分类', 'ABC价格分类', 'ABC需求数量分类',
       '零件号第四位', '组装包', '是否预测']]


In [14]:
load_data.columns = ['HOSTPARTID', 'HOSTLOCID', 'HISTORYBEGDATE', 'DSHOSTID',
       'HISTORYAMOUNT', 'ABC_HITS_NETWORK', 'ABC_COST', 'ABC_DEMAND', 'AA',
       'BB', 'CC']

In [29]:
test = load_data.head(1000000)

In [33]:
# 1. 筛选出demods_details里替代件的历史记录
# 2. 根据仓库代码第二位，如果是0，设置工厂代码为1000，如果是1，工厂代码设置为1100
# 3. 根据工厂代码+零件号 merge匹配库存值，是否继续采购信息，如果库存匹配不上，设置库存为0
# 4. 对于每一行，如果计划员代码非011，032（是否继续采购==1）,那么CPD和RDC都会继续采购/补充老零件，不替代
# 5. 如果计划员代码011，032（是否继续采购==0），有两种情况：
#     a. 库存>0,继续采购/补充老零件，不替代
#     b. 库存<=0,RDC采购转为采购\补充新零件

datetime.datetime(2023, 11, 24, 12, 5, 19, 132798)